In [116]:
import pandas as pd

In [117]:
df_awarie = pd.read_csv('raport_awarii.csv', encoding='utf-8-sig', sep=";")

In [118]:
df_awarie['Utworzono dnia'] = pd.to_datetime(df_awarie['Utworzono dnia'], errors='coerce', dayfirst=True)

# Add the week number column in "W1", "W2", ... format using ISO week
df_awarie['Tydzień'] = df_awarie['Utworzono dnia'].dt.isocalendar().week.astype(int).apply(lambda x: f"W{x}")
cols = df_awarie.columns.tolist()
cols.insert(0, cols.pop(cols.index('Tydzień')))
df_awarie = df_awarie[cols]
df_awarie

,Tydzień,Zawiadomienie,Rodzaj zawiadomienia,Utworzono dnia,Pocz. zakłóc. (godz.),Koniec zakłóc.(godz.),Przestój,Urządzenie,Lokalizacja funkc.,Lokalizacja,...,Czas przestoju,Jedn. czasu przest.,Wpływ,Kod uszkodzenia,Tekst dot. szkody,Kod przyczyny,Ozn. lokaliz. funkc.,Line functional location,Ozn. lokaliz. funkc..1,Oznaczenie obiektu
0,W1,12476707,PT,2025-01-02,06:30:17,00:00:00,NaN,10117891.0,PLPA-PR-U11-010703-003,UGP11,...,"0,00E+00",H,NaN,NaN,NaN,NaN,MODELATOR RAFFAELLO,PLPA-PR-U11-010703,RAFFAELLO LINIA 2,1 - Wałek czyszczący formy
1,W1,12476708,PT,2025-01-02,06:32:26,00:00:00,NaN,10117907.0,PLPA-PR-U11-010703-003,UGP11,...,"0,00E+00",H,NaN,NaN,NaN,NaN,MODELATOR RAFFAELLO,PLPA-PR-U11-010703,RAFFAELLO LINIA 2,1-Składacz form 1 002104
2,W1,12476709,PT,2025-01-02,06:33:21,00:00:00,NaN,10117921.0,PLPA-PR-U11-010703-003,UGP11,...,"0,00E+00",H,NaN,NaN,NaN,NaN,MODELATOR RAFFAELLO,PLPA-PR-U11-010703,RAFFAELLO LINIA 2,1-Składacz form 2 005066
3,W1,12476881,1P,2025-01-02,08:21:02,00:00:00,X,10116830.0,PLPA-PR-U11-010717,UGP11,...,"0,00E+00",H,1.0,NaN,NaN,NaN,LINIA PAKUJĄCA ERMETICO RAFFAELLO,PLPA-PR-U11,UGP11 RAFFAELLO,1-Maszyna pakująca Ermetico 2 003231
4,W1,12476882,1P,2025-01-02,08:26:36,00:00:00,X,10116831.0,PLPA-PR-U11-010717,UGP11,...,"0,00E+00",H,3.0,NaN,NaN,NaN,LINIA PAKUJĄCA ERMETICO RAFFAELLO,PLPA-PR-U11,UGP11 RAFFAELLO,1-Maszyna pakująca Ermetico 3 003775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,W26,12706144,1P,2025-06-29,04:20:43,05:03:09,X,10132055.0,PLPA-PR-U11-010721,UGP11,...,"7,07E-01",H,3.0,103.0,Zużycie,NaN,LINIA WAŻĄCO-PAKUJĄCA 11 T8 L.21 RAFFAEL,PLPA-PR-U11,UGP11 RAFFAELLO,1-Zespół zamykania
1997,W26,12706501,1P,2025-06-29,16:40:45,00:00:00,NaN,10116588.0,PLPA-PR-U11-010703-005,UGP11,...,"0,00E+00",H,2.0,NaN,NaN,NaN,"LINIE WAŻĄCO-PAKUJACE T15 1,2,3,4 RAFFAE",PLPA-PR-U11-010703,RAFFAELLO LINIA 2,1-Transporter pionowy wyj. linia 1-2
1998,W26,12706415,1P,2025-06-29,10:33:00,00:00:00,X,10115784.0,PLPA-PR-U11-010703-006,UGP11,...,"0,00E+00",H,3.0,NaN,NaN,NaN,"LINIE PAKUJĄCE T1,T3,T4-2,3 RAFFAELLO",PLPA-PR-U11-010703,RAFFAELLO LINIA 2,1-Linia pakowania T1/T3/T4 - 3
1999,W26,12706486,1P,2025-06-29,17:04:37,17:25:18,X,10116830.0,PLPA-PR-U11-010717,UGP11,...,"3,45E-01",H,3.0,102.0,Zacinanie się,NaN,LINIA PAKUJĄCA ERMETICO RAFFAELLO,PLPA-PR-U11,UGP11 RAFFAELLO,1-Maszyna pakująca Ermetico 2 003231


In [119]:
# Extract the 5-digit line number from 'Lokalizacja funkc.' style strings
df_awarie['Line_code_raw'] = df_awarie['Lokalizacja funkc.'].astype(str)

pattern = r'(\d{5})(?:-\d{3})?$'

df_awarie['Linia'] = df_awarie['Line_code_raw'].str.extract(pattern).astype(float).astype('Int64')

df_awarie['Linia']

0       10703
1       10703
2       10703
3       10717
4       10717
        ...  
1996    10721
1997    10703
1998    10703
1999    10717
2000    10703
Name: Linia, Length: 2001, dtype: Int64

In [120]:
from datetime import datetime
# Convert 'Utworzono dnia' to datetime
df_awarie['Data'] = pd.to_datetime(df_awarie['Utworzono dnia'], format='%d.%m.%Y', errors='coerce')

# Extract ISO week (e.g., "W01", "W15", etc.) to match with production data's "Tydzień"
df_awarie['Tydzień'] = df_awarie['Data'].dt.isocalendar().week.astype(str).str.zfill(2)
df_awarie['Tydzień'] = 'W' + df_awarie['Tydzień']

df_awarie['Tydzień']

0       W01
1       W01
2       W01
3       W01
4       W01
       ... 
1996    W26
1997    W26
1998    W26
1999    W26
2000    W27
Name: Tydzień, Length: 2001, dtype: object

In [121]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

production_raw = pd.read_csv('test.csv', encoding='utf-8-sig', sep=";")

# Clean downtime numeric
df_awarie['Czas przestoju'] = (
    df_awarie['Czas przestoju'].astype(str)
    .str.replace(',', '.', regex=False)
    .str.replace(' ', '', regex=False)
)
df_awarie['Czas_przestoju_num'] = pd.to_numeric(df_awarie['Czas przestoju'], errors='coerce').fillna(0.0)

# Aggregate downtime per week & line
downtime_weekly = (
    df_awarie.groupby(['Tydzień', 'Linia'])['Czas_przestoju_num']
    .sum()
    .reset_index()
    .rename(columns={'Czas_przestoju_num': 'Total_downtime_min'})
)

# --- Clean & prepare production data ---
prod = production_raw.copy()
prod['Linia'] = pd.to_numeric(prod['Linia'], errors='coerce').astype('Int64')
prod['Tydzień'] = prod['Tydzień'].astype(str).str.strip()  

# Identify efficiency column
eff_col = [c for c in prod.columns if '% Eff LINIA Obb' in c][0]
prod[eff_col] = (
    prod[eff_col].astype(str)
    .str.replace(',', '.', regex=False)
    .str.replace(' ', '', regex=False)
)
prod['Line_efficiency_pct'] = pd.to_numeric(prod[eff_col], errors='coerce')

prod_subset = prod[['Tydzień', 'Linia', 'Line_efficiency_pct']]

# --- Merge datasets ---
merged = pd.merge(downtime_weekly, prod_subset, on=['Tydzień', 'Linia'], how='left').dropna(subset=['Line_efficiency_pct'])

merged

,Tydzień,Linia,Total_downtime_min,Line_efficiency_pct
0,W01,10703,2481.5520,88.42
1,W01,10717,2504.9710,42.79
2,W02,10703,3516.2340,88.42
3,W02,10709,20.5170,80.04
4,W02,10716,0.6550,79.91
...,...,...,...,...
168,W25,10716,20.3567,78.87
169,W25,10717,40.9410,77.81
171,W26,10703,114.1350,88.42
172,W26,10717,8.8720,77.81
